In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon/

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 13.41 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
!unzip -q "/content/Qualcomm-DL-Hackathon/train/images part-1.zip" -d "/content/Qualcomm-DL-Hackathon/train/"
!unzip -q "/content/Qualcomm-DL-Hackathon/train/images part-2.zip" -d "/content/Qualcomm-DL-Hackathon/train/"

In [3]:
import pandas as pd
import os


## name of folders are changed
if not os.path.exists("/content/Qualcomm-DL-Hackathon/train/images_part-1"):
  os.rename(r"/content/Qualcomm-DL-Hackathon/train/images part-1","/content/Qualcomm-DL-Hackathon/train/images_part-1")
if not os.path.exists("/content/Qualcomm-DL-Hackathon/train/images_part-2"):
  os.rename(r"/content/Qualcomm-DL-Hackathon/train/images part-2","/content/Qualcomm-DL-Hackathon/train/images_part-2")

train_csv = r"./Qualcomm-DL-Hackathon/train/train.csv" #"/content/Qualcomm-DL-Hackathon/train/train.csv"
train_df = pd.read_csv(train_csv)

test_csv = r"./Qualcomm-DL-Hackathon/test.csv"
test_df = pd.read_csv(test_csv)

im_pt_1 =  r"./Qualcomm-DL-Hackathon/train/images_part-1" #"/content/Qualcomm-DL-Hackathon/train/images_part-1"
base,_,images = next(os.walk(im_pt_1))

images_path = list(map(lambda im : os.path.join(base,im),images))


im_pt_2 = r"./Qualcomm-DL-Hackathon/train/images_part-2" #"/content/Qualcomm-DL-Hackathon/train/images_part-2"
base,_,images2 = next(os.walk(im_pt_2))

images2_path = list(map(lambda im : os.path.join(base,im),images2))

images.extend(images2)
images_path.extend(images2_path)

image_path_lookup = dict(zip(images,images_path))

paths = []
for r in train_df.iterrows():
  im = r[1].iloc[0]
  pth = image_path_lookup.get(im,None)
  #print(im,pth)
  if not pth:
    print(im)
  paths.append(pth)

train_df["paths"] = paths

paths = []
for r in test_df.iterrows():
  im = r[1].iloc[0]
  pth = image_path_lookup.get(im,None)
  #print(im,pth)
  if not pth:
    print(im)
  paths.append(pth)

test_df["paths"] = paths


test_df.head()

,image_names,paths
0,1960.jpg,./Qualcomm-DL-Hackathon/train/images_part-2/19...
1,668.jpg,./Qualcomm-DL-Hackathon/train/images_part-1/66...
2,2082.jpg,./Qualcomm-DL-Hackathon/train/images_part-2/20...
3,808.jpg,./Qualcomm-DL-Hackathon/train/images_part-1/80...
4,1907.jpg,./Qualcomm-DL-Hackathon/train/images_part-2/19...


In [4]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

image = Image.open(r"./Qualcomm-DL-Hackathon/train/images_part-2/1503.jpg")
image.size

(224, 224)

In [5]:
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):

        self.image_paths = dataframe["paths"].to_list()
        self.labels = dataframe["emergency_or_not"].to_list()
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        image_name = img_path.split("/")[-1]

        if self.transform:
            image = self.transform(image)

        return image_name,image,float(label)


class CustomTestImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):

        self.image_paths = dataframe["paths"].to_list()
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if idx >= len(self.image_paths):
            print(idx)
            raise IndexError
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        image_name = img_path.split("/")[-1]

        if self.transform:
            image = self.transform(image)

        return image_name,image

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# transform = transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = CustomImageDataset(train_df, transform=transform)
#test_dataset = CustomImageDataset(test_df, transform=transform)
batch_size = 15
validation_split = 0.3
shuffle_dataset = True
random_seed= 42

dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = DataLoader(train_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


test_dataset = CustomTestImageDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 1
batch_size = 32
learning_rate = 0.001
num_epochs = 20
# Define CNN model
class CNN(nn.Module):
    def __init__(self, num_classess=num_classes):
        super(CNN, self).__init__()
        self.features = models.resnet18(pretrained=True)
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)


    def forward(self, x):
        x = self.features(x)
        x = nn.functional.sigmoid(x)
        return x.squeeze(1)

model = CNN(num_classes).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
def train(model, loader, criterion, optimizer, device):
    model.train()
    for epoch in range(num_epochs):
        for _,images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.type(torch.DoubleTensor)
            # Forward pass
            outputs = model(images).type(torch.DoubleTensor)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
def evaluate(model, loader, device):
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        for _,images, labels in loader:
            images, labels = images.to(device), labels.to(device).type(torch.IntTensor)
            outputs = model(images)
            predicted = torch.round(outputs).type(torch.IntTensor)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy on test dataset: {accuracy:.2f}%')

# Run training and evaluation
train(model, train_loader, criterion, optimizer, device)
evaluate(model, validation_loader, device)

# Save the model
torch.save(model.state_dict(), 'vehicle_classification_model.pth')


Epoch [1/20], Loss: 0.6071
Epoch [2/20], Loss: 0.3619
Epoch [3/20], Loss: 0.2882
Epoch [4/20], Loss: 0.3456
Epoch [5/20], Loss: 0.0608
Epoch [6/20], Loss: 0.2569
Epoch [7/20], Loss: 0.3438
Epoch [8/20], Loss: 0.0532
Epoch [9/20], Loss: 0.0317
Epoch [10/20], Loss: 0.2518
Epoch [11/20], Loss: 0.2532
Epoch [12/20], Loss: 0.0062
Epoch [13/20], Loss: 0.0047
Epoch [14/20], Loss: 0.0683
Epoch [15/20], Loss: 0.0075
Epoch [16/20], Loss: 0.2697
Epoch [17/20], Loss: 0.0009
Epoch [18/20], Loss: 0.0830
Epoch [19/20], Loss: 0.0514
Epoch [20/20], Loss: 0.0173
Accuracy on test dataset: 91.68%


In [18]:
net = CNN()
net.load_state_dict(torch.load("/content/vehicle_classification_model.pth", weights_only=True))
net.eval()


records = {"image_names" : [],"emergency_or_not" : []}
with torch.no_grad():
    for data in test_loader:
        imname,images = data
        # calculate outputs by running images through the network
        outputs = net(images)
        #print(labels,outputs)
        predicted = torch.round(outputs).type(torch.IntTensor)
        #print(labels,predicted)
        records["image_names"].extend(imname)
        records["emergency_or_not"].extend(predicted.tolist())

sub_df = pd.DataFrame.from_dict(records)
sub_df.to_csv("my_submission.csv",index=False)